In [0]:
import numpy as np
RANDOM_STATE = 2019
np.random.seed(RANDOM_STATE)

from google.colab import drive
drive.mount('/gdrive')

import pandas as pd
import time
import lightgbm as lgb

from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error
from sklearn.model_selection import KFold
from scipy import stats

import gc
import os

os.chdir("/gdrive/My Drive/Colab Notebooks/dacon")

def inverse_boxcox(y, lambda_):
    return np.exp(y) if lambda_ == 0 else np.exp(np.log(lambda_ * y + 1) / lambda_)


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
resampled = pd.read_csv('./data/resampled_0825.csv',parse_dates=['date'])
resampled.drop('holiday',axis=1,inplace=True)
resampled.shape

naive_store_info = pd.read_csv('data/comp_9th/naive_store_info.csv')
naive_store_info = naive_store_info[['store_id','type_bus','high_region']]
resampled = resampled.merge(naive_store_info, on=['store_id'])

In [0]:
resampled = resampled[(resampled.Dayofweek == 3) | (resampled.Dayofweek == 0) | (resampled.Dayofweek == 4)]

In [0]:
resampled.drop(['cycle_idx_week1','cycle_idx_week4','cycle_idx_week8','hday_name','is_weekend','is_holiday','is_weekend','cur_zero'], axis=1,inplace = True)

In [0]:
temp = resampled[['date','store_id','Dayofweek','y']].copy()

def modify_y(dow):
    if dow == 3:
        return 0
    elif dow < 3:
        i = 3 - dow
        return i
    else:
        i = 10 - dow
        return i
    
temp['date_sum'] = temp['Dayofweek'].apply(modify_y)
temp['date_sum'] = pd.to_timedelta(temp['date_sum'],unit='d')
temp['date_thurs'] = temp['date'] + temp['date_sum']

thurs_y = temp[temp.Dayofweek == 3 ][['date','store_id','y']]
thurs_y.columns = ['date_thurs','store_id','new_y']

temp = temp.merge(thurs_y, how = 'left' , on = ['date_thurs','store_id' ]) 
temp = temp[['date','store_id','new_y']]
resampled = resampled.merge(temp,how = 'left', on= ['date','store_id'] )

resampled.drop('y',axis=1,inplace=True)
resampled = resampled.rename(columns={'new_y':'y'})
del temp
import gc
gc.collect()

85

In [0]:
# need to write data and read it again for the same result for some reason

resampled.to_csv('./resampled_check.csv',index=False)

In [0]:
resampled = pd.read_csv('./resampled_check.csv',parse_dates=['date'])

In [0]:
resampled = resampled[(resampled.Dayofweek == 3)]

In [0]:
resampled.shape

(264281, 114)

In [0]:
target = 'y'
cat_vars = ['store_id',  'Month', 'Week', 'Day', 'Dayofweek','Dayofyear',
            'weekofyear', 'type_bus', 'high_region']  

cont_vars = [cols for cols in resampled.columns.values if cols not in cat_vars+[target,'date']]

ratios = [i for i in cont_vars if "ratio" in i]
non_ratios = [i for i in cont_vars if i not in ratios]

In [0]:
for v in non_ratios:
    resampled[v][resampled[v] < 0] = 0

resampled[ratios] = resampled[ratios].replace(np.inf, 1e+17)
resampled[ratios] = resampled[ratios].replace(-np.inf, -1e+17)
resampled[non_ratios] = resampled[non_ratios].fillna(value=-1)
resampled[ratios] = resampled[ratios].fillna(value=0)
print(resampled[cont_vars].columns[resampled[cont_vars].isna().any()].tolist())

#resampled = resampled.set_index('date')
resampled[cat_vars] = resampled[cat_vars].fillna(value='NaN')

from sklearn.preprocessing import LabelEncoder
encoders = {}

for v in cat_vars:
    le = LabelEncoder()
    le.fit(resampled[v].values)
    encoders[v] = le
    resampled.loc[:, v] = le.transform(resampled[v].values)
    # print('{0}: {1}'.format(v, le.classes_))
gc.collect()


[]


28

In [0]:
real_train = resampled[resampled.date <= pd.to_datetime('2018-11-29')]
real_train = real_train[real_train.y > 0]

real_test = resampled[(resampled.date <= pd.to_datetime('2019-02-28')) &
                      (resampled.date >= pd.to_datetime('2019-02-22')) ]

In [0]:

original_target = real_train.y.values
target, lambda_prophet = stats.boxcox(real_train['y'] + 1)
len_train=target.shape[0]
merged_df = pd.concat([real_train, real_test])
merged_df.drop(['y','date'],axis=1,inplace=True)

cate = ['store_id',  'Month', 'Week', 'Day', 'Dayofweek','Dayofyear',
            'weekofyear', 'type_bus', 'high_region']  

params = {
      'num_threads': 4,
      'max_depth': 10,  # 0
      'task': 'train',
      'num_leaves': 511  , 
      'learning_rate': 0.10,    
      'feature_fraction':  0.9, 
      'lambda_l1': 0.06	,       
      'lambda_l2': 0.1,         
      'bagging_fraction':0.8, 
      'bagging_freq': 30	,      
      'max_bin': 64,           
      'metric':'mae',
      'boosting_type': 'gbdt',
      'objective': 'regression_l1',
      'verbose': -1,
      'seed': RANDOM_STATE
      }

test_x = merged_df[len_train:].values
all_x = merged_df[:len_train].values
all_y = target 
oof_preds = np.zeros([all_y.shape[0]])
sub_preds = np.zeros([test_x.shape[0]])
feature_importance_df = pd.DataFrame()

num_folds = 3
num_boost_round = 5000

In [0]:

folds = KFold(n_splits=num_folds, shuffle=True, random_state=RANDOM_STATE)
model_dir = "./"

for n_fold, (train_idx, valid_idx) in enumerate(folds.split(all_x)):
    start_time = time.time()
    train_x, train_y = all_x[train_idx], all_y[train_idx]
    valid_x, valid_y = all_x[valid_idx], all_y[valid_idx]
    lgb_train = lgb.Dataset(train_x,train_y)
    lgb_valid = lgb.Dataset(valid_x,valid_y)
    # train
    gbm = lgb.train(params, lgb_train, num_boost_round= num_boost_round, 
                    categorical_feature = [merged_df.columns.get_loc(c) for c in cate],
                    valid_sets=[lgb_train, lgb_valid],
                    early_stopping_rounds=100, verbose_eval=500)
    
    oof_preds[valid_idx] = gbm.predict(valid_x, num_iteration=gbm.best_iteration)
    sub_preds[:] += gbm.predict(test_x, num_iteration=gbm.best_iteration) / folds.n_splits
    valid_idx += 1

    # if to_submit:
    #     gbm.save_model(model_dir+file_type+str(n_fold)+'_model.txt',
    #                   num_iteration=gbm.best_iteration)
    
    importance_df = pd.DataFrame()
    importance_df['feature'] = merged_df.columns
    importance_df['importance'] = gbm.feature_importance()
    importance_df['fold'] = n_fold + 1
    feature_importance_df = pd.concat([feature_importance_df, importance_df], axis=0)

    print("--- %s seconds ---" % (time.time() - start_time))
    gc.collect()

sub_preds = inverse_boxcox(sub_preds , lambda_prophet) - 1
oof_preds = inverse_boxcox(oof_preds , lambda_prophet) - 1

gc.collect()

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 101, 102, 103, 104, 105, 106, 110, 111]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[500]	training's l1: 2.13424	valid_1's l1: 3.54446
[1000]	training's l1: 1.74644	valid_1's l1: 3.45918
[1500]	training's l1: 1.55997	valid_1's l1: 3.4175
[2000]	training's l1: 1.44201	valid_1's l1: 3.3936
[2500]	training's l1: 1.36586	valid_1's l1: 3.37889
[3000]	training's l1: 1.30645	valid_1's l1: 3.36762
[3500]	training's l1: 1.26136	valid_1's l1: 3.35938
[4000]	training's l1: 1.22463	valid_1's l1: 3.35302
[4500]	training's l1: 1.19397	valid_1's l1: 3.34668
[5000]	training's l1: 1.16902	valid_1's l1: 3.34277
Did not meet early stopping. Best iteration is:
[5000]	training's l1: 1.16902	valid_1's l1: 3.34277
--- 875.4380829334259 seconds ---
Training until validation scores don't improve for 100 rounds.
[500]	training's l1: 2.12378	valid_1's l1: 3.53363
[1000]	training's l1: 1.73525	valid_1's l1: 3.44554
[1500]	training's l1: 1.55086	valid_1's l1: 3.40625
[2000]	training's l1: 1.43768	valid_1's l1: 3.38484
[2500]	training'

0

In [0]:
# Training until validation scores don't improve for 100 rounds.
# [500]	training's l1: 2.13424	valid_1's l1: 3.54446
# [1000]	training's l1: 1.74644	valid_1's l1: 3.45918
# [1500]	training's l1: 1.55997	valid_1's l1: 3.4175
# [2000]	training's l1: 1.44201	valid_1's l1: 3.3936
# [2500]	training's l1: 1.36586	valid_1's l1: 3.37889
# [3000]	training's l1: 1.30645	valid_1's l1: 3.36762
# [3500]	training's l1: 1.26136	valid_1's l1: 3.35938
# [4000]	training's l1: 1.22463	valid_1's l1: 3.35302
# [4500]	training's l1: 1.19397	valid_1's l1: 3.34668
# [5000]	training's l1: 1.16902	valid_1's l1: 3.34277
# Did not meet early stopping. Best iteration is:
# [5000]	training's l1: 1.16902	valid_1's l1: 3.34277
# --- 1074.3606488704681 seconds ---
# Training until validation scores don't improve for 100 rounds.
# [500]	training's l1: 2.12378	valid_1's l1: 3.53363
# [1000]	training's l1: 1.73525	valid_1's l1: 3.44554
# [1500]	training's l1: 1.55086	valid_1's l1: 3.40625
# [2000]	training's l1: 1.43768	valid_1's l1: 3.38484
# [2500]	training's l1: 1.35683	valid_1's l1: 3.36898
# [3000]	training's l1: 1.2993	valid_1's l1: 3.3578
# [3500]	training's l1: 1.25235	valid_1's l1: 3.34845
# [4000]	training's l1: 1.21574	valid_1's l1: 3.34198
# [4500]	training's l1: 1.18379	valid_1's l1: 3.33577
# [5000]	training's l1: 1.15803	valid_1's l1: 3.33141
# Did not meet early stopping. Best iteration is:
# [5000]	training's l1: 1.15803	valid_1's l1: 3.33141
# --- 1027.4489116668701 seconds ---
# Training until validation scores don't improve for 100 rounds.
# [500]	training's l1: 2.10903	valid_1's l1: 3.50743
# [1000]	training's l1: 1.71732	valid_1's l1: 3.41822
# [1500]	training's l1: 1.5329	valid_1's l1: 3.37939
# [2000]	training's l1: 1.42629	valid_1's l1: 3.35927
# [2500]	training's l1: 1.34971	valid_1's l1: 3.3453
# [3000]	training's l1: 1.29134	valid_1's l1: 3.33272
# [3500]	training's l1: 1.24498	valid_1's l1: 3.324
# [4000]	training's l1: 1.2088	valid_1's l1: 3.31741
# [4500]	training's l1: 1.1786	valid_1's l1: 3.31122
# [5000]	training's l1: 1.15168	valid_1's l1: 3.30606
# Did not meet early stopping. Best iteration is:
# [5000]	training's l1: 1.15168	valid_1's l1: 3.30606

In [0]:
store_id_inversed = encoders['store_id'].inverse_transform(real_test.store_id.values)

predicted = pd.DataFrame(
    {'store_id': store_id_inversed,
     'amount': sub_preds
    }
)



In [0]:
predicted.head()

,store_id,amount
0,0,2.124990e+06
1,1,2.979171e+05
2,2,1.056749e+06
3,4,2.833896e+06
4,5,8.921625e+05


In [0]:
feature_importance_df.groupby('feature').sum().sort_values('importance',ascending = False).head(30)

,importance,fold
feature,,
Dayofyear,296606,6
store_id,292734,6
amount_01_ratio,98567,6
amount_12_ratio,96740,6
amount_45_ratio,92418,6
amount_34_ratio,90805,6
amount_23_ratio,90282,6
amount_67_ratio,89257,6
amount_1213_ratio,88291,6


In [0]:
feature_importance_df.to_csv('sub_0830_FeatureImportance.csv',index=False)

In [0]:
predicted.to_csv('./sub_0830_check.csv',index=False)

- ref: 
    - https://machinelearningmastery.com/multi-step-time-series-forecasting/
    - https://www.kaggle.com/adityaecdrid/my-first-time-series-comp-added-prophet
    - https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html
    - https://www.kaggle.com/slivka83/eda-futures-lightgbm-catboost#4.-Model
